<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Data Analysis</center></h1>
<h2><center>Seminar: Neural Networks 2</center></h2>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

import numpy
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

RND_SEED = 7
plt.style.use('ggplot')

numpy.random.seed(RND_SEED)

%matplotlib inline

In [ ]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

from keras.layers import Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense

from keras import backend as K

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

n_train, img_rows, img_cols = X_train.shape
n_test, _, _ = X_test.shape

n_train, n_test, img_rows, img_cols

Посмотрим на данные

## Готовим данные
Есть некоторые заморочки связанные с тем, в каком виде должны быть данные

In [ ]:
X_train.shape

In [ ]:
img_rows, img_cols = 28, 28

if K.image_dim_ordering() != 'tf':
    shape_ord = (1, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 1)

In [ ]:
X_train = X_train.reshape((X_train.shape[0],) + shape_ord)
X_test = X_test.reshape((X_test.shape[0],) + shape_ord)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255


# numbers 0-9, so ten classes
n_classes = 10

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [ ]:
X_train.shape

## Составляем модель

In [ ]:
# Количество конволюционных фильтров
n_filters = 32

# Размер фильра
n_conv = 3

# Размер пуллинга
n_pool = 2

model = Sequential()
model.add(Convolution2D( 
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=shape_ord,
        activation='relu'
))

# Конволюция
model.add(Convolution2D(n_filters, n_conv, n_conv, activation='relu'))

# Пуллинг
model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))
model.add(Dropout(0.25))

# Выравниваем
model.add(Flatten())

# Делаем полносвязный слой
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Softmax в конце
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train,
          y_train,
          batch_size=32,
          nb_epoch=10,
          validation_data=(X_test, y_test),
          verbose=1)

## Внитри модели

In [ ]:
for i, layer in enumerate(model.layers):
    print ("Layer", i, "\t", layer.name, "\t\t", layer.input_shape, "\t", layer.output_shape)

In [ ]:
for i, layer in enumerate(model.layers):
    if len(layer.get_weights()) > 0:
        W, b = layer.get_weights()
        print("Layer", i, "\t", layer.name, "\t\t", W.shape, "\t", b.shape)

## Autoencoders

Постарайтесь составить автокодировщик со следующими настройками

* Слой с 4 конволюционными фильтрами $3 \times 3$
* Max Pooling $2\times 2$
* Слой с 8 конволюционными фильтрами $3 \times 3$
* Max Pooling $2\times 2$
* Слой с 16 конволюционными фильтрами $3 \times 3$
* Up Sampling $2\times 2$
* Слой с 8 конволюционными фильтрами $3 \times 3$
* Up Sampling $2\times 2$
* Слой с 4 конволюционными фильтрами $3 \times 3$
* Up Sampling $2\times 2$
* Слой с 1 конволюционным фильтрами $3 \times 3$



* Подумайте, какие должны быть функции активации.
* Для того, чтобы конфолюция не уменьшала размерность используйте опцию `border_mode='same'`

In [ ]:
from keras.layers.convolutional import UpSampling2D
from keras.layers import Activation

In [ ]:
# Описываем модель автоэнкодер
model2 = Sequential()
# Энкодер
model2.add(...)
# Декодер
model2.add(...)

# Компилируем модель
model2.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
model2.summary()

In [ ]:
# Обучим модель
model2.fit(...)

In [ ]:
preds2 = model2.predict(X_test)

In [ ]:
# Посмотрим, что получилось
c = 1
for image in X_test[:5]:
    plt.subplot(2, 5, c)
    plt.axis('off')
    plt.imshow(image[:,:,0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Test true')
    c += 1
for image in preds2[:5]:
    plt.subplot(2, 5, c)
    plt.axis('off')
    plt.imshow(image[:,:,0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Test true')
    c += 1
plt.show()

# Цветные картинки

In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

nb_classes = 10
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

По сути это уменьшенные изображения, которые нам повсюду встречаются. Сеть должна уметь выполнять классификацию не зависимо от того, повернут ли объект, уменьшен он или увеличен и тп.

То есть в обучающей выборке должны содержаться такие элементарные трансформации

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

generated_images = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

generated_images.fit(X_train)

Далее, продолжаем по https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py